In [ ]:
!pip install yfinance
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pip install plotly>=4.7.1
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

Mounted at /content/drive
/usr/local/bin/orca: Text file busy
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgtk2.0-0 is already the newest version (2.24.32-1ubuntu1).
libgconf-2-4 is already the newest version (3.2.6-4ubuntu1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.9).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
import tensorflow as ft
import os
import os.path
import pandas as pd
import seaborn
from datetime import datetime
from plotly.subplots import make_subplots
from tqdm import tqdm
import numpy as np
from PIL import Image
from numpy import asarray
import sys

In [ ]:
# stock 불러오기

stock_name = input("Write your stock name: ")
stock = yf.Ticker(stock_name)

stock_history = stock.history(period="5d", interval="15m")
# print(stock_history)

df_stock = stock_history

Write your stock name: SPY


In [ ]:
# Candle-stick 이미지 만들기

img_size= 350
data = [[0] for _ in range(50)]
df_open=[]
df_high=[]
df_low=[]
df_close=[]
df_blank = [i for i in range(50)]

for i in range(50):
    df_open.append(df_stock['Open'][len(df_stock)-(4*i)-1])
    df_high.append(df_stock['High'][len(df_stock)-(4*i)-1]) 
    df_low.append(df_stock['Low'][len(df_stock)-(4*i)-1]) 
    df_close.append(df_stock['Close'][len(df_stock)-(4*i)-1]) 
    
fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Candlestick(x=df_blank,
                open=df_open,
                high=df_high,
                low=df_low,
                close=df_close,
                increasing_line_color='red',
                decreasing_line_color='blue',
                showlegend=False),
            row=1, col=1)

fig.add_trace(go.Scatter(x=df_blank,
                y=df_close,
                mode="lines",
                line_color='black',
                showlegend=False),
             row=1, col=1)

fig.update_layout(height=img_size, 
                  width=img_size, 
                  xaxis_rangeslider_visible=False, 
                  plot_bgcolor="white")

fig.layout.xaxis.color = 'white'
fig.show()

In [ ]:
# CNN input

fig.write_image("./img.png")
input_CNN = []

image = Image.open('./img.png').convert('RGB')
os.remove('./img.png')
img_data = np.asarray(image)
img_data = img_data/255.0
input_CNN.append(img_data)
input_CNN = np.array(input_CNN).reshape(1,img_size,img_size,3)

In [ ]:
# LSTM input

input_LSTM=[]
for i in range(50):
    input_LSTM.append(np.log10(df_close[i]))

input_LSTM = np.array(input_LSTM).reshape(1,50,1)

In [ ]:
#모델 정의

#import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
class ResidualUnit(tf.keras.layers.Layer):
  def __init__(self, filters, strides=1, activation ='relu', **kwargs):
    super().__init__(**kwargs)
    self.activation = tf.keras.activations.get(activation)
    self.filters = filters
    self.strides = strides
    self.main_layers = [
                        tf.keras.layers.Conv2D(filters, 3, strides=strides, padding='same', use_bias =False),
                        tf.keras.layers.BatchNormalization(),
                        self.activation,
                        tf.keras.layers.Conv2D(filters, 3, strides=1, padding='same', use_bias = False),
                        tf.keras.layers.BatchNormalization()
    ]
    self.skip_layers =[
                       tf.keras.layers.Conv2D(filters, 1, strides=strides, use_bias = False),
                       tf.keras.layers.BatchNormalization()
    ]

    def get_config(self):
        config = super().get_config()
        config.update({
            "activation": self.activation,
            "strides": self.strides,
            "filters": self.filters,
        })
        return config

    def call(self, inputs):
      Z = inputs
      for layers in self.main_layers:
        Z = layer(Z)
      
      skip_Z = inputs
      for layer in self.skip_layers:
        skip_Z = layer(skip_Z)
      return self.activation(Z + skip_Z)


inputs_CNN = tf.keras.Input(shape=(img_size,img_size,3))

conv1_layer = tf.keras.layers.Conv2D(64, 7, strides=2, input_shape=[img_size, img_size, 3], padding = 'same', use_bias=False)(inputs_CNN)
max_pool_layer1 = tf.keras.layers.MaxPool2D(
        pool_size=(3, 3), padding='VALID', strides=(2, 2),
        name="MaxPooling"
    )(conv1_layer)
BN1 = tf.keras.layers.BatchNormalization()(max_pool_layer1)
Relu1 = tf.keras.layers.Activation('relu')(BN1)
max_pool_layer2 = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2)(Relu1)
prev_filters = 64


ResnetList = []
ResnetList.append(ResidualUnit(filters = 64, strides= 1)(max_pool_layer2))
for i, filters in enumerate([64]*2 + [128]*4 + [256] *6 + [512]*3):
  strides = 1 if filters == prev_filters else 2
  ResnetList.append(ResidualUnit(filters = filters, strides= strides)(ResnetList[i]))
  prev_filters = filters

  

extractLayer = tf.keras.layers.Conv2D(512, 1, strides=1, input_shape=[87, 87, 512], padding = 'same', use_bias=False)(ResnetList[-1])
GA = tf.keras.layers.GlobalAveragePooling2D()(extractLayer)
flatten_CNN = tf.keras.layers.Flatten()(GA)
from tensorflow.keras import layers, models
def create_lstm_model(log_inputs: tf.keras.Input) -> layers.Layer:
    lstm_layer_1 = layers.LSTM(50, batch_input_shape=(None, 50, 1), return_sequences=True)(log_inputs)
    lstm_layer_2 = layers.LSTM(50, batch_input_shape=(None, 50, 50))(lstm_layer_1)
    flatten_layer = layers.Flatten()(lstm_layer_2)
    full_connected_1 = layers.Dropout(0.5)(
        layers.Dense(500, activation='relu', use_bias=True)(flatten_layer)
    )
    full_connected_2 = layers.Dropout(0.5)(
        layers.Dense(100, activation='relu', use_bias=True)(full_connected_1)
    )
    full_connected_3 = layers.Dropout(0.5)(
        layers.Dense(25, activation='relu', use_bias=True)(full_connected_2)
    )
    output_layer = layers.Dense(1, activation='linear')(full_connected_3)
    return output_layer, flatten_layer

#input들만 넣을 수 있으면 된다.

inputs_LSTM = tf.keras.Input(shape=(50, 1))

_, cell_output = create_lstm_model(inputs_LSTM)

combined_feature = layers.Concatenate(
            axis=1)([cell_output, flatten_CNN])
flatten_layer = layers.Flatten()(combined_feature)

full_connected_1 = layers.Dropout(0.5)(
            layers.Dense(500, activation='relu', use_bias=True)(flatten_layer)
        )
full_connected_2 = layers.Dropout(0.5)(
            layers.Dense(100, activation='relu',
                         use_bias=True)(full_connected_1)
        )
full_connected_3 = layers.Dropout(0.5)(
            layers.Dense(25, activation='relu',
                         use_bias=True)(full_connected_2)
        )
lstm_cnn_output = layers.Dense(
            1, activation='linear')(full_connected_3)

lstm_cnn_model = models.Model(
            inputs=[inputs_LSTM, inputs_CNN], outputs=lstm_cnn_output
        )

In [ ]:
### Model & Weight 불러오는 코드 추가하기 ###
# 모델 가져오기

model = models.Model(
            inputs=[inputs_LSTM, inputs_CNN], outputs=lstm_cnn_output
        )
model.load_weights('/content/drive/MyDrive/Capstone2/code/models/Fusion_S&P500_welldone.h5')


In [ ]:
# Predict & Calculate Ratio of surge/plunge

pred_stock_price = model.predict([input_LSTM, input_CNN])
latest_stock_price = input_LSTM[0,-1,0]


In [ ]:
print(10**(pred_stock_price - (0.0568)*(1+0.0313)), 10**latest_stock_price)
#0.031324107090254766 [0.05682444] 10**(predictions + (1+deviation)*gap_avg
stock_ratio = (10**(pred_stock_price - (0.0568)*(1+0.0313)) - 10**latest_stock_price)/(10**latest_stock_price)*100
print(stock_ratio)
# stock_ratio > 0 : surge
# stock_ratio < 0 : plunge

[[484.71405]] 470.9800109863283
[[2.9160557]]


In [ ]:
predictions = lstm_cnn_model.predict([X_test_LSTM, X_test_CNN])
predictions=np.array(predictions)

gap_ratio = []
gap_sign = []
for i in range(len(Y_test_LSTM)):
  gap_ratio.append(abs(Y_test_LSTM[i] - predictions[i])) 
  gap_sign.append(Y_test_LSTM[i] - predictions[i])
gap_avg = sum(gap_ratio) / len(Y_test_LSTM)
sign =  sum(gap_sign)
deviation = np.std(gap_sign)


plt.figure(figsize=(21, 9))
plt.plot(10**Y_test_LSTM, label = 'actual')
if sign > 0:
  plt.plot(10**(predictions + gap_avg*(1+deviation)), label = 'prediction')
else:
  plt.plot(10**(predictions - gap_avg*(1-deviation)), label = 'prediction')
plt.legend()
plt.show()

In [ ]:
### Percentage for alert - input으로 입력 받아서 예측된 stock_ratio와 비교 ###